In [6]:
%pip install -U sentence-transformers starpoint llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.1-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.83-cp311-cp311-macosx_13_0_arm64.whl size=333902 sha256=b90553e9158e1a06a12561d743df0a2139d661864f8ed6a8fcbe7a56d43112e1
  Stored in directory: /Users/sw/Library/Caches/pip/wheels/25/d2/5b/9f3c919284f260835ba686d041a70e06ae7c35adc62493188e
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


In [8]:
!mkdir -p model
!wget https://huggingface.co/TheBloke/Airoboros-L2-7B-2.1-GGUF/resolve/main/airoboros-l2-7b-2.1.Q5_K_M.gguf -O model/model.gguf

--2023-08-30 14:13:04--  https://huggingface.co/TheBloke/Airoboros-L2-7B-2.1-GGUF/resolve/main/airoboros-l2-7b-2.1.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.244.214.111, 18.244.214.57, 18.244.214.53, ...
Connecting to huggingface.co (huggingface.co)|18.244.214.111|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/55/ca/55ca22016aea4434ae89cb0c4c7ba8702ccd11446f25a31715f506292a9e8b71/0586a817d16520f07ae6260626bc0f67c995bf281f358945d8b956289242be54?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27airoboros-l2-7b-2.1.Q5_K_M.gguf%3B+filename%3D%22airoboros-l2-7b-2.1.Q5_K_M.gguf%22%3B&Expires=1693689184&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MzY4OTE4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy81NS9jYS81NWNhMjIwMTZhZWE0NDM0YWU4OWNiMGM0YzdiYTg3MDJjY2QxMTQ0NmYyNWEzMTcxNWY1MDYyOTJhOWU4YjcxLzA1ODZhODE3ZDE2NTIwZjA3YWU2Mj

In [ ]:
from sentence_transformers import SentenceTransformer

embeddings_data = defaultdict(dict)

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for key, texts in monster_texts.items():
    for text_field, text in texts.items():
        embedding = model.encode(text)
        embeddings_data[key][text_field] = embedding

In [ ]:
import copy
from starpoint.db import Client

COLLECTION_NAME = 'dnd'
API_KEY = '2a3ed27a-ced2-4ad6-b411-511a5c71f6e3'

documents_to_upload = []

for key, embeddings in embeddings_data.items():
    metadata = monsters[key]
    for text_field, embedding in embeddings.items():
        new_metadata = copy.deepcopy(metadata)
        new_metadata['embedding_source'] = text_field
        documents_to_upload.append({
            'metadata': new_metadata,
            'embedding': embedding.tolist()
        })

client = Client(api_key=API_KEY)
client.insert(documents=documents_to_upload, collection_name=COLLECTION_NAME)


{'collection_id': '7eda4d2c-1f99-44ff-bc0f-5f2d49ce7397',
 'documents': [{'id': 'wiyf9mubdc2l'},
  {'id': 'bomfi3kl5w5o'},
  {'id': 'reezxivjjy1t'},
  {'id': 'nvrqud37f436'},
  {'id': 'pfmihaewo0wz'},
  {'id': 'uli0yvtqb2jn'},
  {'id': 'eybti67jla5v'},
  {'id': 'znbi01h7yljm'},
  {'id': 'azl1ddcmj5lo'},
  {'id': 'my71n0iqwf56'},
  {'id': 'jwtk2u7e9vyx'},
  {'id': 'qtkr5lawrgwj'},
  {'id': 'lcgs2i4la810'},
  {'id': 'yw3kcf4y9v6v'},
  {'id': 'koekpbc0qf5o'},
  {'id': 'lxqtmfhoghny'},
  {'id': 'j4wpb4lh8bb7'},
  {'id': 'eh5veln7qmpb'},
  {'id': '300wgkyem48v'},
  {'id': 'auqzy2q8zxfo'},
  {'id': 'my1uvtp58r9v'},
  {'id': '2zx7b6h36dof'},
  {'id': 'bqy4irksnw5a'},
  {'id': 'ddfdo9nty29j'},
  {'id': '0i3vzehelq8i'},
  {'id': 'v5lob1gsn386'},
  {'id': 'snpmolx0v5rj'},
  {'id': '7xzxzblmj5yg'},
  {'id': '5lh0o7f728k3'},
  {'id': 't0kt0vkqw91c'},
  {'id': 'kpxww79feetg'},
  {'id': 'bqd1xy27sana'},
  {'id': 'eflo7m65gsgf'},
  {'id': 'teepicfr34pn'},
  {'id': 'p2addjaewx6p'},
  {'id': '8xs5j5s34

In [ ]:
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
from starpoint.db import Client

COLLECTION_NAME = 'dnd'
API_KEY = '2a3ed27a-ced2-4ad6-b411-511a5c71f6e3'
starpoint_client = Client(api_key=API_KEY)
GGML_MODEL_PATH = "./model/model.bin"

llm = Llama(model_path=GGML_MODEL_PATH)

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def query_starpoint(text):
    query_embedding = model.encode(text).tolist()
    relevant_monsters = starpoint_client.query(
        collection_id=COLLECTION_NAME,
        query_embedding=query_embedding,
        sql="SELECT * FROM collection LIMIT 5"
    )
    return relevant_monsters


In [ ]:
# rerun this cell to initialize or reset the chat history
chat_history = []

In [ ]:
resp = quers 